***
## notebook config

In [1]:
ON_COLAB = False
ASSESS = True

In [2]:
if ON_COLAB:
    from google.colab import drive
    from google.colab import files
    drive.mount('/content/drive', force_remount=True)
    
    !pip install --upgrade kaggle > /dev/null 2>&1
    !mkdir -p ~/.kaggle/ && cp /content/drive/MyDrive/kaggle/kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json
    
    !free -h
    
    !pip install --upgrade category_encoders > /dev/null 2>&1
    !pip install --upgrade tsforest > /dev/null 2>&1
    !pip install --upgrade lightgbm > /dev/null 2>&1
    !pip install --upgrade optuna > /dev/null 2>&1

In [3]:
if ON_COLAB:
    !kaggle datasets download -d mavillan/meli-2021 --force --unzip
    !ls -halt
    input_path = "./"
    print("input_path:", input_path)
    subs_path = "/content/drive/MyDrive/meli2021/subs"
    print("subs_path:", subs_path)
    results_path = "/content/drive/MyDrive/meli2021/results"
    print("results_path:", results_path)
else:
    input_path = "../data"
    print("input_path:", input_path)
    subs_path = "../subs"
    print("subs_path:", subs_path)
    results_path = "../results"
    print("results_path:", results_path)

input_path: ../data
subs_path: ../subs
results_path: ../results


***

In [4]:
from copy import deepcopy
import gc
import dill
from glob import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import time
from tqdm import tqdm
import yaml
import shap
import optuna

from sklearn import preprocessing
import category_encoders as ce
import lightgbm as lgb
from tsforest.forecast import LightGBMForecaster

import sys
sys.path.append("../utils")
from memory import reduce_mem_usage
from metrics import RMSSE, ranked_probability_score, scoring_function, binarize_predictions
from inventory import InventoryDaysPredictor
from features import Featurador

In [5]:
SEED = 19
np.random.seed(SEED)
pd.set_option('display.max_columns', None)

***
## Data preparation

In [6]:
with open(f"../data/skus_assess_m1.yaml", "r") as file:
    skus_assess_m1 = yaml.load(file, Loader=yaml.FullLoader)
    print(f"len(skus_assess_m1): {len(skus_assess_m1)}")
    file.close()

with open(f"../data/skus_assess_m2.yaml", "r") as file:
    skus_assess_m2 = yaml.load(file, Loader=yaml.FullLoader)
    print(f"len(skus_assess_m2): {len(skus_assess_m2)}")
    file.close()
    
skus_for_test = pd.read_csv(f"{input_path}/test_data.csv").sku.values
print(f"len(skus_for_test): {len(skus_for_test)}")

len(skus_assess_m1): 482635
len(skus_assess_m2): 272130
len(skus_for_test): 551472


In [7]:
scales = pd.read_csv(f"{input_path}/scales.csv")

dataset = (
    pd.read_parquet(f"{input_path}/train-m1.parquet")
    .query("sku in @skus_assess_m1")
    .rename({"date":"ds", "sold_quantity":"y"}, axis=1,)
    .sort_values(["sku","ds"])
    .reset_index(drop=True)
)

with open("../encoders/encoder-stg1.dill", "rb") as file:
    encoder = dill.load(file)
    file.close()

transformed = encoder.transform(dataset[encoder.cols].astype("category"))
for col in transformed.columns:
    dataset[col+"_glm"] = transformed[col].values

In [8]:
raw = pd.read_parquet(f"{input_path}/train_data.parquet", columns=["sku", "date", "sold_quantity", "minutes_active"])
metadata = pd.read_csv(f"{input_path}/metadata.csv", usecols=["sku","item_domain_id","site_id"])
raw = pd.merge(raw, metadata, how="inner", on="sku")

# compute q_mean and q_std features (w/o leakage)
feat = Featurador(raw)
feat.fit(left_limit="2021-02-01", right_limit="2021-03-01")
dataset = feat.transform(dataset)

In [9]:
raw = raw.query("'2021-02-01' <= date <= '2021-03-01'").reset_index(drop=True)
raw["had_sales"] = raw.eval("sold_quantity > 0")
raw["was_active"] = raw.eval("minutes_active > 120")

sales_rate = (raw.groupby("sku")["had_sales"].sum() / raw.groupby("sku")["date"].count()).reset_index(name="sales_rate")
activity_rate = (raw.groupby("sku")["was_active"].sum() / raw.groupby("sku")["date"].count()).reset_index(name="activity_rate")

dataset = (
    dataset
    .merge(sales_rate, how="inner", on="sku")
    .merge(activity_rate, how="inner", on="sku")
)

***
## model config

In [10]:
categorical_features = {
    "listing_type": "default",
    "shipping_logistic_type": "default",
    "shipping_payment": "default",
    "item_domain_id": "default",
    "item_domain_id_glob": "default",
    "site_id":"default",
    "product_id_glob": "default",
    "product_family_id_glob": "default",
}

exclude_features = [
    "fold",
    "sold_quantity",
    "sku",
    "item_id",
    "minutes_active",
    "product_id",
    "product_id_glm",
    "product_family_id",
    "product_family_id_glm",
]

model_params = {
    'objective':'tweedie',
    'boosting':'dart',
    'drop_rate':0.1,
    'max_drop':20,
    'skip_drop':0.66,
    'extra_trees': True,
    'metric':'None',
    'max_bin': 127,
    'bin_construct_sample_cnt':30000000,
    'num_leaves': 2**8-1,
    'min_data_in_leaf': 2**8-1,
    'learning_rate': 0.01,
    'max_delta_step':1.,
    'feature_fraction':0.8,
    'bagging_fraction':0.8,
    'bagging_freq':1,
    'lambda_l2':0.1,
    'max_cat_threshold': 64,
    'cat_l2': 50,
    'cat_smooth': 50,
    'boost_from_average': False,
    'force_row_wise': True,
    #'deterministic':True,
    'seed':SEED,
    'num_threads':16,
}
time_features = [
    "week_day",
    "week_day_cos",
    "week_day_sin",
    "month_progress",
]
model_kwargs = {
    "model_params":model_params,
    "time_features":time_features,
    "exclude_features":exclude_features,
    "categorical_features":categorical_features,
    "ts_uid_columns":["sku",],
    #"lags": [1,],
    #"window_functions":{
    #    "mean":   (None, [1,], [3,7,]),
    #    "std":    (None, [1,], [3,7]),
    #},
}

***
## Loss tuning: tweedie

In [11]:
all_results = list()

def objective(trial):
    vp = trial.suggest_discrete_uniform("vp", 1, 1.2, 0.1)
    w = trial.suggest_discrete_uniform("w", 0., 0.2, 0.1)
    num_iterations = int(trial.suggest_discrete_uniform("num_iterations", 300, 600, 50))
            
    result = {
        "num_iterations":num_iterations,
        "tweedie_variance_power":vp,
        "min_weight":w,
    }
    
    model_kwargs["model_params"]["tweedie_variance_power"] = vp
    model_kwargs["model_params"]["num_iterations"] = num_iterations
    
    if "weight" in dataset.columns:
        dataset.drop("weight", axis=1, inplace=True)
    if w < 1:
        scaler = preprocessing.MinMaxScaler(feature_range=(w,1))
        dataset["weight"] = scaler.fit_transform(dataset["minutes_active"].values.reshape(-1,1)).ravel()
        
    # training the model
    train_data = dataset.copy()
    valid_idx = train_data.query("ds > '2021-03-01'").index

    model = LightGBMForecaster(**model_kwargs)
    model.prepare_features(train_data, valid_idx)
    model.train_features = reduce_mem_usage(model.train_features, verbose=False)
    model.valid_features = reduce_mem_usage(model.valid_features, verbose=False)
    gc.collect()

    evaluator = RMSSE(train_data.loc[valid_idx, ["sku","y"]], scales)
    model.fit(fit_kwargs={"verbose_eval":100, "feval":evaluator.evaluate})
    result["rmsse"] = model.model.model.best_score["valid_0"]["rmsse"]

    # generating oof predictions
    oof = model.predict(dataset.query("ds > '2021-03-01'"), recursive=False)
    
    # generates the ID predictions
    predictor = InventoryDaysPredictor(dataset.query("ds <= '2021-03-01'"))
    predictor.fit(oof)
    
    valid = (
        pd.read_csv(f"{input_path}/validation_seed2_harder.csv")
        .query("sku in @skus_assess_m1")
        .reset_index(drop=True)
    )
    
    preds = list()
    for sku,df in tqdm(valid.groupby("sku")):
        preds.append(predictor.predict(sku, df.target_stock.values[0])[0])   
    valid["days_to_stockout"] = np.asarray(preds)
    valid["days_to_stockout_disc"] = valid["days_to_stockout"].astype(int)
    
    rmse1 = np.sqrt(valid.query("inventory_days <= 31").eval("(days_to_stockout - inventory_days)**2").mean())
    mean_ae1 = valid.query("inventory_days <= 31").eval("abs(days_to_stockout - inventory_days)").mean()
    median_ae1 = valid.query("inventory_days <= 31").eval("abs(days_to_stockout - inventory_days)").median()
    
    result["rmse1"] = rmse1
    result["mean_ae1"] = mean_ae1
    result["median_ae1"] = median_ae1
    
    rmse2 = np.sqrt(valid.query("sku in @skus_assess_m2").eval("(days_to_stockout - inventory_days)**2").mean())
    mean_ae2 = valid.query("sku in @skus_assess_m2").eval("abs(days_to_stockout - inventory_days)").mean()
    median_ae2 = valid.query("sku in @skus_assess_m2").eval("abs(days_to_stockout - inventory_days)").median()
    
    result["rmse2"] = rmse2
    result["mean_ae2"] = mean_ae2
    result["median_ae2"] = median_ae2
    
    all_results.append(result)
    
    del model,evaluator,predictor,valid,train_data
    gc.collect()
    
    return mean_ae1

In [12]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=10000, timeout=21600, n_jobs=1) # 6-hrs

[I 2021-09-09 16:24:56,211] A new study created in memory with name: no-name-a297789a-a6ac-413c-805a-35bb679592d8


[LightGBM] [Warning] boosting is set=dart, boosting_type=gbrt will be ignored. Current value: boosting=dart
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbrt will be ignored. Current value: boosting=dart
[LightGBM] [Info] Total Bins 41276
[LightGBM] [Info] Number of data points in the train set: 13893795, number of used features: 23
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbrt will be ignored. Current value: boosting=dart
[100]	valid_0's rmsse: 0.98623
[200]	valid_0's rmsse: 0.929393
[300]	valid_0's rmsse: 0.901502
[400]	valid_0's rmsse: 0.885751
[500]	valid_0's rmsse: 0.881034


100%|██████████| 482635/482635 [01:18<00:00, 6151.40it/s]
[I 2021-09-09 17:15:37,630] Trial 0 finished with value: 7.07659525976262 and parameters: {'vp': 1.0, 'w': 0.1, 'num_iterations': 500.0}. Best is trial 0 with value: 7.07659525976262.


[LightGBM] [Warning] boosting is set=dart, boosting_type=gbrt will be ignored. Current value: boosting=dart
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbrt will be ignored. Current value: boosting=dart
[LightGBM] [Info] Total Bins 41276
[LightGBM] [Info] Number of data points in the train set: 13893795, number of used features: 23
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbrt will be ignored. Current value: boosting=dart
[100]	valid_0's rmsse: 0.985308
[200]	valid_0's rmsse: 0.932541
[300]	valid_0's rmsse: 0.90958


100%|██████████| 482635/482635 [01:18<00:00, 6120.01it/s]
[I 2021-09-09 17:51:22,698] Trial 1 finished with value: 7.176667622389376 and parameters: {'vp': 1.1, 'w': 0.0, 'num_iterations': 350.0}. Best is trial 0 with value: 7.07659525976262.


...

100%|██████████| 482635/482635 [01:19<00:00, 6095.24it/s]
[I 2021-09-09 20:55:18,771] Trial 5 finished with value: 7.087651868699886 and parameters: {'vp': 1.2, 'w': 0.2, 'num_iterations': 500.0}. Best is trial 3 with value: 7.054012158694941.


[LightGBM] [Warning] boosting is set=dart, boosting_type=gbrt will be ignored. Current value: boosting=dart
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbrt will be ignored. Current value: boosting=dart
[LightGBM] [Info] Total Bins 41276
[LightGBM] [Info] Number of data points in the train set: 13893795, number of used features: 23
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbrt will be ignored. Current value: boosting=dart
[100]	valid_0's rmsse: 0.981186
[200]	valid_0's rmsse: 0.921632
[300]	valid_0's rmsse: 0.892524
[400]	valid_0's rmsse: 0.875384


100%|██████████| 482635/482635 [01:19<00:00, 6100.94it/s]
[I 2021-09-09 21:38:51,524] Trial 6 finished with value: 7.0374632145002165 and parameters: {'vp': 1.0, 'w': 0.2, 'num_iterations': 450.0}. Best is trial 6 with value: 7.0374632145002165.


[LightGBM] [Warning] boosting is set=dart, boosting_type=gbrt will be ignored. Current value: boosting=dart
[LightGBM] [Warning] Met negative value in categorical features, will convert it to NaN
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbrt will be ignored. Current value: boosting=dart
[LightGBM] [Info] Total Bins 41276
[LightGBM] [Info] Number of data points in the train set: 13893795, number of used features: 23
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbrt will be ignored. Current value: boosting=dart
[100]	valid_0's rmsse: 0.966478
[200]	valid_0's rmsse: 0.908486
[300]	valid_0's rmsse: 0.882415
[400]	valid_0's rmsse: 0.868342
[500]	valid_0's rmsse: 0.864387


100%|██████████| 482635/482635 [01:18<00:00, 6141.00it/s]
[I 2021-09-09 22:33:21,844] Trial 7 finished with value: 7.1049188982648745 and parameters: {'vp': 1.2, 'w': 0.2, 'num_iterations': 550.0}. Best is trial 6 with value: 7.0374632145002165.


In [21]:
study.trials_dataframe().drop_duplicates(subset=["params_vp","params_w","params_num_iterations"]).sort_values("value").head(20)

,number,value,datetime_start,datetime_complete,duration,params_num_iterations,params_vp,params_w,state
6,6,7.037463,2021-09-09 20:55:18.773513,2021-09-09 21:38:51.523843,0 days 00:43:32.750330,450.0,1.0,0.2,COMPLETE
3,3,7.054012,2021-09-09 18:35:39.395684,2021-09-09 19:29:53.018465,0 days 00:54:13.622781,550.0,1.0,0.2,COMPLETE
2,2,7.071225,2021-09-09 17:51:22.699704,2021-09-09 18:35:39.393587,0 days 00:44:16.693883,450.0,1.0,0.1,COMPLETE
0,0,7.076595,2021-09-09 16:24:56.213310,2021-09-09 17:15:37.629813,0 days 00:50:41.416503,500.0,1.0,0.1,COMPLETE
5,5,7.087652,2021-09-09 20:05:47.447109,2021-09-09 20:55:18.771546,0 days 00:49:31.324437,500.0,1.2,0.2,COMPLETE
7,7,7.104919,2021-09-09 21:38:51.526048,2021-09-09 22:33:21.844176,0 days 00:54:30.318128,550.0,1.2,0.2,COMPLETE
4,4,7.124045,2021-09-09 19:29:53.020379,2021-09-09 20:05:47.445256,0 days 00:35:54.424877,350.0,1.1,0.1,COMPLETE
1,1,7.176668,2021-09-09 17:15:37.631646,2021-09-09 17:51:22.697669,0 days 00:35:45.066023,350.0,1.1,0.0,COMPLETE


In [22]:
result_df = pd.DataFrame(all_results).drop_duplicates(subset=["tweedie_variance_power","min_weight","num_iterations"])

In [23]:
result_df.sort_values("rmse1").head(10)

,num_iterations,tweedie_variance_power,min_weight,rmsse,rmse1,mean_ae1,median_ae1,rmse2,mean_ae2,median_ae2
6,450,1.0,0.2,0.871534,9.502366,7.037463,5.036918,8.795883,6.563271,4.873756
3,550,1.0,0.2,0.867728,9.519433,7.054012,5.057084,8.790925,6.557622,4.880219
5,500,1.2,0.2,0.864387,9.548337,7.087652,5.100287,8.813073,6.581473,4.919019
2,450,1.0,0.1,0.883020,9.554371,7.071225,5.049917,8.866320,6.613026,4.906474
0,500,1.0,0.1,0.881034,9.559575,7.076595,5.058564,8.861210,6.607682,4.904569
7,550,1.2,0.2,0.863521,9.569171,7.104919,5.122371,8.824514,6.588823,4.926825
4,350,1.1,0.1,0.886961,9.614966,7.124045,5.107184,8.976814,6.713893,5.000000
1,350,1.1,0.0,0.903339,9.691495,7.176668,5.141203,9.063608,6.775188,5.000000


In [24]:
result_df.sort_values("mean_ae1").head(10)

,num_iterations,tweedie_variance_power,min_weight,rmsse,rmse1,mean_ae1,median_ae1,rmse2,mean_ae2,median_ae2
6,450,1.0,0.2,0.871534,9.502366,7.037463,5.036918,8.795883,6.563271,4.873756
3,550,1.0,0.2,0.867728,9.519433,7.054012,5.057084,8.790925,6.557622,4.880219
2,450,1.0,0.1,0.883020,9.554371,7.071225,5.049917,8.866320,6.613026,4.906474
0,500,1.0,0.1,0.881034,9.559575,7.076595,5.058564,8.861210,6.607682,4.904569
5,500,1.2,0.2,0.864387,9.548337,7.087652,5.100287,8.813073,6.581473,4.919019
7,550,1.2,0.2,0.863521,9.569171,7.104919,5.122371,8.824514,6.588823,4.926825
4,350,1.1,0.1,0.886961,9.614966,7.124045,5.107184,8.976814,6.713893,5.000000
1,350,1.1,0.0,0.903339,9.691495,7.176668,5.141203,9.063608,6.775188,5.000000


In [25]:
result_df.sort_values("median_ae1").head(10)

,num_iterations,tweedie_variance_power,min_weight,rmsse,rmse1,mean_ae1,median_ae1,rmse2,mean_ae2,median_ae2
6,450,1.0,0.2,0.871534,9.502366,7.037463,5.036918,8.795883,6.563271,4.873756
2,450,1.0,0.1,0.883020,9.554371,7.071225,5.049917,8.866320,6.613026,4.906474
3,550,1.0,0.2,0.867728,9.519433,7.054012,5.057084,8.790925,6.557622,4.880219
0,500,1.0,0.1,0.881034,9.559575,7.076595,5.058564,8.861210,6.607682,4.904569
5,500,1.2,0.2,0.864387,9.548337,7.087652,5.100287,8.813073,6.581473,4.919019
4,350,1.1,0.1,0.886961,9.614966,7.124045,5.107184,8.976814,6.713893,5.000000
7,550,1.2,0.2,0.863521,9.569171,7.104919,5.122371,8.824514,6.588823,4.926825
1,350,1.1,0.0,0.903339,9.691495,7.176668,5.141203,9.063608,6.775188,5.000000


In [26]:
result_df.sort_values("rmse2").head(10)

,num_iterations,tweedie_variance_power,min_weight,rmsse,rmse1,mean_ae1,median_ae1,rmse2,mean_ae2,median_ae2
3,550,1.0,0.2,0.867728,9.519433,7.054012,5.057084,8.790925,6.557622,4.880219
6,450,1.0,0.2,0.871534,9.502366,7.037463,5.036918,8.795883,6.563271,4.873756
5,500,1.2,0.2,0.864387,9.548337,7.087652,5.100287,8.813073,6.581473,4.919019
7,550,1.2,0.2,0.863521,9.569171,7.104919,5.122371,8.824514,6.588823,4.926825
0,500,1.0,0.1,0.881034,9.559575,7.076595,5.058564,8.861210,6.607682,4.904569
2,450,1.0,0.1,0.883020,9.554371,7.071225,5.049917,8.866320,6.613026,4.906474
4,350,1.1,0.1,0.886961,9.614966,7.124045,5.107184,8.976814,6.713893,5.000000
1,350,1.1,0.0,0.903339,9.691495,7.176668,5.141203,9.063608,6.775188,5.000000


In [27]:
result_df.sort_values("mean_ae2").head(10)

,num_iterations,tweedie_variance_power,min_weight,rmsse,rmse1,mean_ae1,median_ae1,rmse2,mean_ae2,median_ae2
3,550,1.0,0.2,0.867728,9.519433,7.054012,5.057084,8.790925,6.557622,4.880219
6,450,1.0,0.2,0.871534,9.502366,7.037463,5.036918,8.795883,6.563271,4.873756
5,500,1.2,0.2,0.864387,9.548337,7.087652,5.100287,8.813073,6.581473,4.919019
7,550,1.2,0.2,0.863521,9.569171,7.104919,5.122371,8.824514,6.588823,4.926825
0,500,1.0,0.1,0.881034,9.559575,7.076595,5.058564,8.861210,6.607682,4.904569
2,450,1.0,0.1,0.883020,9.554371,7.071225,5.049917,8.866320,6.613026,4.906474
4,350,1.1,0.1,0.886961,9.614966,7.124045,5.107184,8.976814,6.713893,5.000000
1,350,1.1,0.0,0.903339,9.691495,7.176668,5.141203,9.063608,6.775188,5.000000


In [28]:
result_df.sort_values("median_ae2").head(10)

,num_iterations,tweedie_variance_power,min_weight,rmsse,rmse1,mean_ae1,median_ae1,rmse2,mean_ae2,median_ae2
6,450,1.0,0.2,0.871534,9.502366,7.037463,5.036918,8.795883,6.563271,4.873756
3,550,1.0,0.2,0.867728,9.519433,7.054012,5.057084,8.790925,6.557622,4.880219
0,500,1.0,0.1,0.881034,9.559575,7.076595,5.058564,8.861210,6.607682,4.904569
2,450,1.0,0.1,0.883020,9.554371,7.071225,5.049917,8.866320,6.613026,4.906474
5,500,1.2,0.2,0.864387,9.548337,7.087652,5.100287,8.813073,6.581473,4.919019
7,550,1.2,0.2,0.863521,9.569171,7.104919,5.122371,8.824514,6.588823,4.926825
1,350,1.1,0.0,0.903339,9.691495,7.176668,5.141203,9.063608,6.775188,5.000000
4,350,1.1,0.1,0.886961,9.614966,7.124045,5.107184,8.976814,6.713893,5.000000


***